# Generate data for `test_correct_count_rate`

In [8]:
import os
import pprint

from astropy.io import fits
import numpy as np

from jwst_magic.fsw_file_writer import buildfgssteps

In [3]:
__location__ = '/Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/'
ROOT = "test_buildfgssteps"
SEGMENT_INFILE = os.path.join(__location__, 'data', '{}_ALLpsfs.txt'.format(ROOT))
SELECTED_SEGS = os.path.join(__location__, 'data', '{}_regfile.txt'.format(ROOT))

In [4]:
with fits.open(
    '/Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/fgs_data_2_cmimf.fits'
) as hdulist:
    open_image = hdulist[1].data

In [ ]:
1 ID
bias : [4113.857078887622, 104049.3416361762]
norm : [-1.7679394239994082e-20, 5283.94677734375]
strips : [4113.857078887622, 65535.0]
cds : [-16.6, 65000.0]
product : [4113.857078887622, 65535.0]
countrates: [65596. 91475. 91549. 95731. 95747.]

In [31]:
d = {}
for guider in [1, 2]:
    d[guider] = {}
    for step in ['LOSTRK', 'CAL', 'ID', 'ACQ1', 'ACQ2', 'TRK']:
        d[guider][step] = {}
        
        bias = []
        norm = []
        strips = []
        cds = []
        product = []
        
        if step != 'LOSTRK':
            n = 10
        else:
            n = 1
            
        for i in range(n):
            # Run the code
            BFS = buildfgssteps.BuildFGSSteps(open_image, guider, 'test', step, guiding_selections_file=SELECTED_SEGS,
                                out_dir=__location__, catalog=SEGMENT_INFILE);

            # Don't know how to test bias - since it is so variable
            if step is not 'LOSTRK':
                bias.append([np.min(BFS.bias), np.max(BFS.bias)])

            norm.append([np.min(BFS.time_normed_im), np.max(BFS.time_normed_im)])
            if step is 'ID':
                strips.append([np.min(BFS.strips), np.max(BFS.strips)])
            if 'TRK' not in step:
                cds.append([np.min(BFS.cds), np.max(BFS.cds)])
            product.append([np.min(BFS.image), np.max(BFS.image)])
        
        print()
        print(guider, step)
        print()
        for l, name in zip([bias, norm , strips , cds , product, BFS.countrate], 
                              ['bias', 'norm' , 'strips' , 'cds' , 'product', 'countrate']):
            if (l != [] and step == 'LOSTRK') or (name in ['countrate', 'norm']):
                d[guider][step][name] = l[0]
            elif l != [] and name != 'countrate':
#                 print(name, ': [{}, {}]'.format(np.average([m[0] for m in l]), np.average([m[1] for m in l])))
#                 print(name)
#                 print(l)
#                 print()
#                 print(np.average([m[0] for m in l]))
#                 print(np.average([m[1] for m in l]))
                d[guider][step][name] = [round(np.average([m[0] for m in l]), -1), 
                                         round(np.average([m[1] for m in l]), -1)]
#             elif name == 'countrate':
#                 d[guider][step][name] = l
#         print('countrates:', BFS.countrate)
        

2019-03-20 11:38:44,253 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113844_buildfgssteps_test.log
2019-03-20 11:38:44,255 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2019-03-20 11:38:44,260 - INFO - FSW File Writing: Creating LOSTRK FSW files

1 LOSTRK

2019-03-20 11:38:44,320 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113844_buildfgssteps_test.log
2019-03-20 11:38:44,322 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file


/Users/lchambers/miniconda3/envs/magic/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


Adjusting 4509 pixels that are saturated in read 2.
Adjusting 4366 pixels that are saturated in both reads.
2019-03-20 11:38:45,557 - INFO - FSW File Writing: Creating CAL FSW files
2019-03-20 11:38:45,610 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113845_buildfgssteps_test.log
2019-03-20 11:38:45,611 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
Adjusting 4511 pixels that are saturated in read 2.
Adjusting 4365 pixels that are saturated in both reads.
2019-03-20 11:38:46,614 - INFO - FSW File Writing: Creating CAL FSW files
2019-03-20 11:38:46,674 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113846_buildfgssteps_test.log
2019-03-20 11:38:46,675 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-co

/Users/lchambers/miniconda3/envs/magic/lib/python3.7/site-packages/ipykernel_launcher.py:39: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


Successfully wrote: /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/out/test/shifted/all_found_psfs_test_G1.txt
Successfully wrote: /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/out/test/shifted/test_G1.fits
2019-03-20 11:38:57,163 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
Adjusting 4182 pixels that are saturated in read 2.
Adjusting 4182 pixels that are saturated in both reads.
2019-03-20 11:39:00,005 - INFO - FSW File Writing: Creating ID FSW files
2019-03-20 11:39:00,159 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113900_buildfgssteps_test.log
2019-03-20 11:39:00,213 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)
Successfully wrote: /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magi

...

Adjusting 2 pixels that are saturated in read 2.
Adjusting 2 pixels that are saturated in both reads.
2019-03-20 11:39:39,181 - INFO - FSW File Writing: Creating ACQ2 FSW files
2019-03-20 11:39:39,191 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113939_buildfgssteps_test.log
2019-03-20 11:39:39,193 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
Adjusting 2 pixels that are saturated in read 2.
Adjusting 2 pixels that are saturated in both reads.
2019-03-20 11:39:39,215 - INFO - FSW File Writing: Creating ACQ2 FSW files
2019-03-20 11:39:39,225 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113939_buildfgssteps_test.log
2019-03-20 11:39:39,226 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissionin

2019-03-20 11:39:56,789 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2019-03-20 11:39:58,565 - INFO - FSW File Writing: Creating TRK FSW files
2019-03-20 11:39:58,613 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_113958_buildfgssteps_test.log
2019-03-20 11:39:58,616 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
2019-03-20 11:40:00,521 - INFO - FSW File Writing: Creating TRK FSW files

1 TRK

2019-03-20 11:40:00,575 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_114000_buildfgssteps_test.log
2019-03-20 11:40:00,576 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commission

Adjusting 2483 pixels that are saturated in read 2.
Adjusting 2481 pixels that are saturated in both reads.
2019-03-20 11:40:18,614 - INFO - FSW File Writing: Creating ID FSW files
2019-03-20 11:40:18,761 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_114018_buildfgssteps_test.log
2019-03-20 11:40:18,813 - INFO - FSW File Writing: Shifting guide star to ID attitude (1024, 1024)
Successfully wrote: /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/out/test/shifted/guiding_selections_test_G2.txt
Successfully wrote: /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/out/test/shifted/all_found_psfs_test_G2.txt
Successfully wrote: /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/out/test/shifted/test_G2.fits
2019-03-20 11:40:19,884 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_b

Adjusting 2485 pixels that are saturated in read 2.
Adjusting 2483 pixels that are saturated in both reads.
2019-03-20 11:40:41,061 - INFO - FSW File Writing: Creating ID FSW files

2 ID

2019-03-20 11:40:41,188 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_114041_buildfgssteps_test.log
2019-03-20 11:40:41,189 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
Adjusting 16 pixels that are saturated in read 2.
Adjusting 16 pixels that are saturated in both reads.
2019-03-20 11:40:41,236 - INFO - FSW File Writing: Creating ACQ1 FSW files
2019-03-20 11:40:41,295 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_114041_buildfgssteps_test.log
2019-03-20 11:40:41,296 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs

2019-03-20 11:40:41,896 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_114041_buildfgssteps_test.log
2019-03-20 11:40:41,897 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
Adjusting 2 pixels that are saturated in read 2.
Adjusting 2 pixels that are saturated in both reads.
2019-03-20 11:40:41,916 - INFO - FSW File Writing: Creating ACQ2 FSW files
2019-03-20 11:40:41,926 - INFO - Started logging to file /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/logs/2019_03_20_Wed_114041_buildfgssteps_test.log
2019-03-20 11:40:41,927 - INFO - FSW File Writing: Using /Users/lchambers/TEL/Commissioning/Tools/fgs-commissioning/jwst_magic/tests/data/test_buildfgssteps_regfile.txt as the guiding selections file
Adjusting 2 pixels that are saturated in read 2.
Adjusting 2 pixels that are saturated in b

In [32]:
pprint.pprint(d)

{1: {'ACQ1': {'bias': [11920.0, 87850.0],
              'cds': [-20.0, 65000.0],
              'countrate': 65596.0,
              'norm': [0.0039671236, 1977.3064],
              'product': [11920.0, 65540.0]},
     'ACQ2': {'bias': [13940.0, 81660.0],
              'cds': [-10.0, 65000.0],
              'countrate': 65596.0,
              'norm': [0.02249858, 137.2947],
              'product': [13940.0, 65540.0]},
     'CAL': {'bias': [4120.0, 104050.0],
             'cds': [-20.0, 65000.0],
             'countrate': 65596.0,
             'norm': [0.0, 166565.97],
             'product': [4120.0, 65540.0]},
     'ID': {'bias': [4120.0, 104050.0],
            'cds': [-20.0, 65000.0],
            'countrate': 65596.0,
            'norm': [-1.7679394e-20, 5283.9473],
            'product': [4120.0, 65540.0],
            'strips': [4120.0, 65540.0]},
     'LOSTRK': {'countrate': 65596.0,
                'norm': [0.0060026245, 137.2947],
                'product': [6.730232e-05, 1.539368

# Generate data for `coordinate_tranforms`

In [6]:
import inspect
from jwst_magic.utils import coordinate_transforms

In [22]:
for func in sorted(dir(coordinate_transforms)):
    if not func.startswith('_') and '2' in func:
        print(func)
        func_to_call = getattr(coordinate_transforms, func)
#         result = func_to_call()
        arg_names = inspect.getfullargspec(func_to_call)[0]
#         print(arg_names)
    
        if 'x_raw' in arg_names:
            x, y = (1743.3, 241.9)
            
        else:
            x, y = (-55.736935, 8.139518)
            
        if 'guider' in arg_names:
            for guider in range(2):
                result = func_to_call(x, y, guider + 1)
                print(result)
        else:
            result = func_to_call(x, y)
            print(result)

DHAS2Raw
(928, 206)
(1156, 207)
Idl2DHAS
(55.736935, 8.139518)
Idl2Raw
(882, 1833)
(1130, 1847)
Raw2DHAS
(-53.70147089878638, -49.332391547578155)
(-53.14152903705431, 48.846349592719555)
Raw2Det
(241.9, 1743.3)
Raw2Idl
(53.70147089878638, -49.332391547578155)
(53.14152903705431, 48.846349592719555)
Raw2Tel
(-52.6118059444085, -50.49287779246356)
(-52.97899983353878, 49.02258207773389)


In [ ]:
method_to_call = getattr(foo, 'bar')
result = method_to_call()

In [21]:
ANGLE_COORDS = (-55.736935, 8.139518)
coordinate_transforms.DHAS2Raw(*ANGLE_COORDS, 1)

(928, 206)